Importing Libraries

In [5]:
from classiq import *

In [6]:
from classiq.qmod.symbolic import logical_or
from classiq.execution import ExecutionPreferences

4 qubits are needed to represent 16 nodes

In [7]:
size = 4 
num_nodes = 2**size 


Consider a graph (line) with total 16 nodes or vertices denoted as V and edges with only the adjacent one E.

To simulate a quantum walk on a general graph, the state $|j,k⟩$ needs to be acted upon by $SC$ operators. The C operator is,
$C = \sum_{j \in V} |j⟩⟨j| \otimes \left( 2 |∂_j⟩ ⟨∂_j| - I\right) $
with $|∂_j⟩ = \frac{1}{deg(j)} \sum_{k :(j,k)\in E} |k⟩$,
where deg(j) = number of adjacent vertices to the vertex $j$. So, in our setting, except for the end vertices (0,15), all vertices have deg(j) =1/2.

For the implementation of C operator, phase kickback it used, notice
$2 |∂_j⟩ ⟨∂_j| - I = U_j \left(2 |0⟩⟨0| - I \right)U_j^{†}.$
Here $P = 2 |0⟩⟨0| - I $ when acts on any state it behaves as a phase kickback operator,
$\left(2 |0⟩⟨0| - I\right) |x⟩ = (-1)^{x \neq 0} |x⟩$.
The below code implements the needed phase kickback operator $P$.


In [8]:
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

As per the explanation above, C or W_operator is implemented here.

The C operator is,
$C = \sum_{j \in V} |j⟩⟨j| \otimes \left( 2 |∂_j⟩ ⟨∂_j| - I\right) $
with $|∂_j⟩ = \frac{1}{deg(j)} \sum_{k :(j,k)\in E} |k⟩$,
where deg(j) = number of adjacent vertices to the vertex $j$. So, in our setting, except for the end vertices (0,15), all vertices have deg(j) =1/2.


Here, $U_j P U_j^{†}$ is implemented, where $prob$ is used to capture the effect of operator U as $U|0⟩$, throguh inplace_prepare_state.

And within_apply this $U_j P U_j^{†}$ is finally captured.

In [9]:
def C_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    
    num_nodes = 2**4
    prob = [0]*num_nodes  
    if i == 0:
        prob[i + 1] = 1.0 
    elif i == num_nodes - 1:
        prob[i - 1] = 1.0 
    else:
        prob[i - 1] = 0.5  
        prob[i + 1] = 0.5  
    print(f'Node={i}, prob vec ={prob}')
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))

@qfunc 
def C_operator(vertices:QNum, adjacent_vertices: QNum):
    
    num_nodes = 2**4
    for i in range(num_nodes):
        C_iteration(i,vertices,adjacent_vertices)




Now the operator S is just bitwise swap. But requires the details of the edges of the graph. Note that this graph is a simple path or line. So the edge exists if $j-k =1 $ or $j-k =-1$, this is captured in res, in the edge_oracle.

The S operator has the form,
$S = \sum_{(j,k)\in E} |j,k⟩\langle k,j| $.

In [10]:
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices - adjacent_vertices) == 1) | ((vertices - adjacent_vertices) == -1))


@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))
  

@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))



Here, in the main qfunc, vertices and adjacent_vertices are initailised, followed by other steps to implement H on vertices and then W and S operators

In [11]:
@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    size = 4
    allocate(size, vertices)  # Allocate qubits for vertices
    hadamard_transform(vertices)  # Apply Hadamard transform to initialize superposition
    allocate(size, adjacent_vertices)  # Allocate qubits for adjacent vertices

    C_operator(vertices, adjacent_vertices)  # Apply coin operator
    S_operator(vertices, adjacent_vertices)  # Apply shift operator

In [13]:
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Node=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
Node=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
Node=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
Node=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
Node=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
Node=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
Node=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
Node=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]
Node

/var/folders/v4/gng44qs519q3hzsgrnlm91d40000gn/T/ipykernel_51676/1207959271.py:13: DeprecationWarning: Parameter 'operand' of function 'control' has been renamed to 'stmt_block'. Parameter 'operand' will be deprecated in a future release.
Hint: Change `control(ctrl=..., operand=...)` to `control(ctrl=..., stmt_block=...)` or `control(..., ...)`.
  control(ctrl=vertices==i,
/var/folders/v4/gng44qs519q3hzsgrnlm91d40000gn/T/ipykernel_51676/1207959271.py:14: DeprecationWarning: Parameters 'compute' and 'action' of function 'within_apply' have been renamed to 'within' and 'apply' respectively. Parameters 'compute' and 'action' will be deprecated in a future release.
Hint: Change `within_apply(compute=..., action=...)` to `within_apply(within=..., apply=...)` or `within_apply(..., ...)`.
  operand=lambda: within_apply(
/var/folders/v4/gng44qs519q3hzsgrnlm91d40000gn/T/ipykernel_51676/4050587547.py:16: DeprecationWarning: Parameters 'compute' and 'action' of function 'within_apply' have been r

Opening: https://platform.classiq.io/circuit/dfeaac9a-d8e3-4417-a78f-ef6ccd246524?version=0.46.0


I used the syntax according to the previous version, and hence faced some errors in the end, however the matrix obtained is c